In [11]:
import os
import pandas as pd
import numpy as np
from src.data.ksea_ztest import ksea_ztest

In [12]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_logician/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_logician/')

In [40]:
# selecting cell line
cline = 'HL60'  # HL60, MCF7, NTERA2
mode = ''  # '', 'nas_', 'rd_nas_'
percentile = 99.5

In [41]:
# Load observational data and enz_sub relationships
# Has to be repeated for every cell line individually
obs = pd.read_csv('data/raw/ctamdb_dpoa_' + mode + cline + '.tsv', sep='\t')
enz_sub_omnipath = pd.read_csv('data/processed/facts/enz_sub_omnipath.csv')
enz_sub_cantley = pd.read_csv('data/processed/facts/enz_sub_cantley.csv')
kinases = pd.read_csv('data/processed/facts/kinase.csv')
phosphatases = pd.read_csv('data/processed/facts/phosphatase.csv')

In [42]:
# filter enz_sub_cantley data based on percentile column
enz_sub_cantley = enz_sub_cantley[enz_sub_cantley['percentile'] > percentile]
# merge enzu_sub relationships
enz_sub = pd.concat([enz_sub_omnipath, enz_sub_cantley], ignore_index=True)

In [43]:
enz_sub

,enzyme,substrate,kpa,kpa_class,tprot,pst,residue_type,residue_offset,modification,sources,references,n_resources,n_references,percentile
0,Q2M2I8,Q9BXS5,AAK1,K,AP1M1,AP1M1(T144),T,144.0,phosphorylation,HPRD_MIMP;MIMP;PhosphoSite_MIMP;ProtMapper;SIG...,11877461,4.0,1.0,NaN
1,Q2M2I8,Q96CW1,AAK1,K,AP2M1,AP2M1(T154),T,154.0,phosphorylation,HPRD_MIMP;MIMP;PhosphoSite_MIMP;phosphoELM_MIMP,NaN,1.0,0.0,NaN
2,Q2M2I8,Q96CW1,AAK1,K,AP2M1,AP2M1(T156),T,156.0,phosphorylation,PhosphoSite;PhosphoSite_ProtMapper;ProtMapper;...,11877457;11877461,4.0,2.0,NaN
3,Q2M2I8,P49757,AAK1,K,NUMB,NUMB(T102),T,102.0,phosphorylation,MIMP;PhosphoSite;PhosphoSite_MIMP;PhosphoSite_...,18657069;25904845,5.0,2.0,NaN
4,P00519,Q8IZP0,ABL1,K,ABI1,ABI1(Y213),Y,213.0,phosphorylation,HPRD_MIMP;MIMP;PhosphoSite;PhosphoSite_MIMP;Ph...,17389395;18083107;18328268;19534553;20598684;2...,5.0,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47985,NaN,NaN,ATM,NaN,TMEM51,TMEM51(S115),NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
47986,NaN,NaN,MYLK4,NaN,ATR,ATR(S435),NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
47987,NaN,NaN,GRK5,NaN,NAP1L4,NAP1L4(S125),NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
47988,NaN,NaN,GRK7,NaN,PCNP,PCNP(S119),NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0


In [44]:
# Data wrangling INPUT
# Reformat observations and enz_sub data
kt = enz_sub[['kpa', 'pst']].drop_duplicates()
obs = obs[['perturbagen', 'pst', 'fc']]
# Data is converted into df, where rows and cols correspond to phosphosite and perturbagens, respectively.
obs = obs.pivot(index='pst', columns='perturbagen', values='fc')
obs = obs.replace(np.nan, 'nan')

In [45]:
kt

,kpa,pst
0,AAK1,AP1M1(T144)
1,AAK1,AP2M1(T154)
2,AAK1,AP2M1(T156)
3,AAK1,NUMB(T102)
4,ABL1,ABI1(Y213)
...,...,...
47985,ATM,TMEM51(S115)
47986,MYLK4,ATR(S435)
47987,GRK5,NAP1L4(S125)
47988,GRK7,PCNP(S119)


In [46]:
# kseaZtest function returns Z-scores and p-values for every phosphosite and perturbagen
ksea_df = ksea_ztest(kt, obs, graphics = False, min_sub = 1)

In [47]:
len(ksea_df.kpa.unique())

475

In [48]:
# Assign activity status
ksea_df['status'] = 'unaffected'
ksea_df.loc[ksea_df['n_sub_data'] == 0, 'status'] = 'no_t'  # no observed targets
ksea_df.loc[(ksea_df['kpa'].isin(kinases['kinase'])) & (ksea_df['z_score'] > 0) & (
            ksea_df['p_val'] < 0.05), 'status'] = 'actv'
ksea_df.loc[(ksea_df['kpa'].isin(kinases['kinase'])) & (ksea_df['z_score'] < 0) & (
            ksea_df['p_val'] < 0.05), 'status'] = 'inhb'
ksea_df.loc[(ksea_df['kpa'].isin(phosphatases['phosphatase'])) & (ksea_df['z_score'] < 0) & (
            ksea_df['p_val'] < 0.05), 'status'] = 'actv'
ksea_df.loc[(ksea_df['kpa'].isin(phosphatases['phosphatase'])) & (ksea_df['z_score'] > 0) & (
            ksea_df['p_val'] < 0.05), 'status'] = 'inhb'

In [49]:
ksea_df

,kpa,pert,p_val,n_sub_data,mlog2fc,z_score,n_sub_db,status
0,AAK1,AC220,0.457885,16,0.026441,0.105764,19,unaffected
1,AAK1,AT13148,0.266862,16,-0.155583,-0.622333,19,unaffected
2,AAK1,AZ20,0.305968,16,-0.126828,-0.507311,19,unaffected
3,AAK1,AZD1480,0.258157,16,0.162259,0.649038,19,unaffected
4,AAK1,AZD3759,0.278232,16,-0.147026,-0.588102,19,unaffected
...,...,...,...,...,...,...,...,...
28970,ZAK,Torin,0.307838,41,0.078397,0.501987,42,unaffected
28971,ZAK,Trametinib,0.424050,41,-0.029914,-0.191544,42,unaffected
28972,ZAK,U73122,0.003693,41,0.418375,2.678905,42,unaffected
28973,ZAK,Ulixertinib,0.043239,41,0.267725,1.714279,42,unaffected


In [24]:
ksea_df.value_counts('status')

status
unaffected    24585
inhb           3209
actv           1120
dtype: int64

In [51]:
# export df to csv
ksea_df.to_csv('data/processed/facts/ksea_' + mode + cline + '_mp_cantley.csv', index=False)

In [52]:
# Write Prolog file
with open('models/facts/ksea_' + mode + cline + '_mp_cantley.pl', 'w') as file:
    for index, row in ksea_df.iterrows():
        var1 = "ksea('{}', '{}', {}, {}).".format(row['pert'], row['kpa'], row['status'], row['p_val'])
        file.write(var1 + '\n')